## **Análise dos métodos de detecção de Data drift da biblioteca Deepchecks**

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sns1

### **Feature Drift**

#### **Característica alvo: "Nenhum"**

In [2]:
enade_treino = pd.read_csv("../../concept_drift/tabela_final_2017_treinamento.csv")
print(enade_treino["Numero_Participantes"].mean())
print(len(enade_treino["Numero_Participantes"]))

0.829518283954788
1605


In [3]:
enade_teste = pd.read_csv("../../concept_drift/tabela_final_2021_treinamento.csv")
print(enade_teste["Numero_Participantes"].mean())
print(len(enade_teste["Numero_Participantes"]))

0.7650466057487378
1506


### **ECDF** (Empirical Cumulative Distribution Function)

An empirical distribution function is the function associated with the empirical measure of a sample. This cumulative distribution function is a step function that jumps up by 1/n at each of the n data points. Its value at any **specified value** of the measured variable is **the fraction of observations** of the measured variable that are **less than or equal to the specified value**.

No contexto do Enade, temos que o ECDF apresenta a fração $\mathcal{Y}$ de cursos de computação que apresentam uma porcentagem menor ou igual a $\mathcal{X}$ de alunos que escolheram a alternativa "Nenhum" para uma determinada questão.

Pergunta: Se não tivéssemos feito a transformação dos valores para porcentagens, estaria tudo certo? Ou transformar para porcentagens foi necessário? A normalização faria mesma coisa? Como a normalização faz parte do Pipeline de treinamento do modelo e não da formação do conjunto em si, talvez a transformação como operação canônica para formação do conjunto tem seu valor?

- Forma antiga de calcular o **ECDF** utilizando a biblioteca Seaborn:

In [ ]:
teste = sns.ecdfplot(enade_teste["Nenhum"]).get_lines()[0].get_data()
treino = sns1.ecdfplot(enade_treino["Nenhum"]).get_lines()[0].get_data()

In [ ]:
data_points_teste = sns.ecdfplot(enade_teste["Nenhum"]).get_lines()[0].get_data()[0]
data_points_treino = sns.ecdfplot(enade_treino["Nenhum"]).get_lines()[0].get_data()[0]

In [ ]:
common_set = set(data_points_teste) & set(data_points_treino)

- Forma antiga de calcular a métrica **K-S test**:

In [ ]:
# Calculate the K-S test metric for two CDF 'teste' e 'treino' on the measured variable "Nenhum"

max = 0

for i in common_set:
    print(i)
    index_teste = np.where(teste[0] == i)[0][0]
    print(index_teste[0][0])
    index_treino = np.where(treino[0] == i)[0][0]
    print(index_treino)
    
    diff = teste[1][index_teste] - treino[1][index_treino]
    
    if diff < 0:
        diff = diff * -1
   
    if diff > max:
        max = diff

print(max)

- Nova forma de calcular o **ECDF**:

In [4]:
# Create a sorted series of unique data (from the measured variable "Nenhum")
# The unique operation is done for the sake of the evenly spaced sequence operation below
# The sorting is for the probability calculation of each x_values in the loop
data_points = np.sort(enade_treino["Numero_Participantes"].unique())

# X-data for the ECDF: evenly spaced sequence of the uniques
# By doing this (evenly spaced sequence), the graph is smooth until the end
# Step function (for the evenly spaced sequence): (stop - start)/(num - 1)
x_values = np.linspace(start=min(data_points), stop=max(data_points),num=len(data_points))

In [5]:
# Total number of observations
data_size_treino = enade_treino["Numero_Participantes"].size

# Y-data for the ECDF:
y_values_treino = []

for i in x_values:
    # All the values in raw data less than the ith value in x_values
    temp = enade_treino["Numero_Participantes"][enade_treino["Numero_Participantes"] <= i]
    
    # Fraction of that range of values with respect to the total number of observations
    value = temp.size / data_size_treino
    
    # Pushing the fraction (cumulative probability of the ith value in x_values) in the y_values
    y_values_treino.append(value)

In [6]:
# Total number of observations
data_size_teste = enade_teste["Numero_Participantes"].size

# Y-data for the ECDF:
y_values_teste = []

for i in x_values:
    # All the values in raw data less than the ith value in x_values
    temp = enade_teste["Numero_Participantes"][enade_teste["Numero_Participantes"] <= i]
    
    # Fraction of that range of values with respect to the total number of observations
    value = temp.size / data_size_teste
    
    # Pushing the fraction (cumulative probability of the ith value in x_values) in the y_values
    y_values_teste.append(value)

In [7]:
# Calculate the K-S test metric for two CDF 'teste' e 'treino' on the measured variable "Nenhum"

max = 0

for i in range(0, len(x_values)):
    diff = y_values_teste[i] - y_values_treino[i]
    
    if diff < 0:
        diff = diff * -1
   
    if diff > max:
        max = diff

print(max)

0.2159883829169304


In [ ]:
_ = plt.plot(x_values, y_values_treino, marker='.', linestyle='none')
_ = plt.xlabel('Percentage of students who choose the \'Nenhum\' option')
_ = plt.ylabel('Percentage of computer courses')
plt.margins(0.02)